In [1]:
import stockdata as sd
import pandas as pd
import os
import yfinance as yf
import scipy.optimize as spo
import numpy as np

In [2]:
period = '6mo'

In [3]:
spy = sd.normalize(yf.Ticker('SPY').history(period=period)['Close'])

In [4]:
spydr = sd.calc_dr(spy)

In [5]:
data = sd.fetch_data(period, 'sp500.txt')

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- CTL: No data found, symbol may be delisted
- MYL: No data found, symbol may be delisted


In [6]:
sd.fill_gaps(spy)

In [7]:
sd.fill_gaps(data)

In [8]:
close = sd.get_close(data)

In [9]:
for date in close.index:
    if date not in spy.index:
        close.drop(date, inplace=True)

In [10]:
dr = sd.calc_dr(close)

In [11]:
def find_fit(xSer, ySer):
        if not xSer.index.equals(ySer.index):
            raise Exception('Regression: Indexes do not match')
        xBar = xSer.mean() #SPY mean
        yBar = ySer.mean() #Other mean
        xyBar = xSer.multiply(ySer).mean()
        x2Bar = (xSer ** 2).mean()
        xBar2 = xBar ** 2
        m = (xBar * yBar - xyBar) / (xBar2 - x2Bar) #beta
        yInt = yBar - m * xBar #alpha
        return m, yInt

In [12]:
greek = pd.DataFrame(columns = ['Alpha', 'Beta'])

In [13]:
for ticker in dr.columns.values:
    beta, alpha = find_fit(spydr, dr[ticker])
    greek.loc[ticker] = [alpha, beta]

In [14]:
greek.sort_values(by=['Alpha'],axis=0, ascending=False, inplace=True)

In [16]:
greek.head(20)

,Alpha,Beta
TSLA,0.006605,1.963870
TPR,0.006448,0.687966
VIAC,0.005019,0.353135
IVZ,0.004957,1.075949
DISCA,0.004909,0.633638
GM,0.004794,0.938253
MOS,0.004699,0.899493
FCX,0.004595,1.411964
SIVB,0.004398,0.952551
ALB,0.004221,1.412281
